# Gibbs energy governed fatigue model

This notebook uses the same hypothesis as the previous one to produce animations of the fatigue behavior. 

In [ ]:
%matplotlib widget
from bmcs_matmod import GSM
import matplotlib.pylab as plt 
import sympy as sp
from bmcs_utils.api import Cymbol
import numpy as np
from bmcs_matmod.gsm.potentials import Potential1D_T_E_VP_D_SymbExpr
sp.init_printing()

In [ ]:
# p1d = Potential1D_T_E_VP_D_SymbExpr()
# eps_a = p1d.eps_a
# sig_a = p1d.sig_a
# dF_du = p1d.F_.diff(eps_a)
# dF_du = dF_du.xreplace({h: 0 for h in dF_du.atoms(sp.DiracDelta)})
# dF_du = dF_du.xreplace({h: 1 for h in dF_du.atoms(sp.Heaviside)})
# u_sig_ = sp.Matrix([ sp.solve(sp.Eq(sig_i, dF_du_i), u_i)[0] for sig_i, u_i, dF_du_i in 
#                             zip(sig_a, eps_a, dF_du)])
# subs_u_sig_ = dict(zip(eps_a, u_sig_))
# subs_u_sig_

## Gibbs free energy

In [ ]:
p1d = Potential1D_T_E_VP_D_SymbExpr()
eps_a = p1d.eps_a
sig_a = p1d.sig_a
dF_du = p1d.F_.diff(eps_a)
dF_du = dF_du.xreplace({h: 0 for h in dF_du.atoms(sp.DiracDelta)})
dF_du = dF_du.xreplace({h: 1 for h in dF_du.atoms(sp.Heaviside)})
u_sig_ = sp.Matrix([ sp.solve(sp.Eq(sig_i, dF_du_i), u_i)[0] for sig_i, u_i, dF_du_i in 
                            zip(sig_a, eps_a, dF_du)])
subs_u_sig_ = dict(zip(eps_a, u_sig_))
subs_u_sig_

In [ ]:
sig_x_u_ = (sig_a.T * eps_a)[0]
G_expr = sig_x_u_ - p1d.F_
G_ = sp.simplify(G_expr.subs(subs_u_sig_))
G_

In [ ]:
gsm_F = GSM(
    u_vars = eps_a,
    sig_vars = sig_a,
    T_var = p1d.T,
    m_params = p1d.mparams,
    Eps_vars = p1d.Eps_vars,
    Sig_vars = p1d.Sig_vars,
    Sig_signs = (-1, 1, 1, -1),
    F_expr = p1d.F_,
    f_expr = p1d.f_,
    phi_ext_expr = p1d.phi_ext_,
    t_relax = p1d.t_relax_
)

gsm_G = GSM(
    u_vars = sig_a,
    sig_vars = eps_a,
    T_var = p1d.T,
    m_params = p1d.mparams,
    Eps_vars = p1d.Eps_vars,
    Sig_vars = p1d.Sig_vars,
    Sig_signs = (1, -1, -1, 1),
    F_expr = G_,
    dF_sign = -1,
    f_expr = p1d.f_,
    phi_ext_expr = p1d.phi_ext_,
    t_relax = p1d.t_relax_
)

In [ ]:
gsm_F.F_expr

In [ ]:
gsm_F.phi_

In [ ]:
gsm_G.F_expr

In [ ]:
sp.simplify(gsm_F.dF_dEps_), sp.simplify(gsm_G.dF_dEps_)

In [ ]:
gsm_F.Sig_vars, gsm_F.Sig_, sp.simplify(gsm_G.Sig_)

In [ ]:
sp.simplify(gsm_F.df_dSig_), sp.simplify(gsm_G.df_dSig_)

In [ ]:
sp.simplify(gsm_F.Phi_), sp.simplify(gsm_G.Phi_)

In [ ]:
sp.simplify(gsm_F.df_dEps_), sp.simplify(gsm_G.df_dEps_)

In [ ]:
gsm_G.phi_, sp.simplify(gsm_G.Phi_)

In [ ]:
gsm_F.dSig_dEps_, sp.simplify(gsm_G.dSig_dEps_)

## Monotonic loading

In [ ]:
sig_ = gsm_F.F_expr.diff(gsm_F.u_vars)
sig_

In [ ]:
_f_c = 44
_f_t = -0.1 * _f_c
_X_0 = (_f_c + _f_t) / 2
_f_s = (_f_c - _f_t) / 2
_E = 50000
_KH_factor = 4
_KH = _E * _KH_factor
_K_ratio = 0.01 # 0.015
_K = _KH * _K_ratio
_H = _KH * (1 - _K_ratio)
material_params = dict(
    E_=_E, 
    gamma_lin_= _H, # _E * 10, 
    gamma_exp_=0.5,
    alpha_0_=0.5,
    K_lin_= _K, # _E / 5,
    k_exp_=10,
    z_0_=10,
    S_=0.008,
    c_=2.5,
    r_=2.7,
    f_c_=_f_s,
    X_0_=_X_0,  
    eta_=500,
    T_0_=20,
    C_v_=0.01, # 0.0001, 
    beta_=0.0001,
    alpha_therm_=0, # 1.2e-5,
    d_N_ = 1
)

In [ ]:
def gsm_run(gsm_, u_ta, T_t, t_t, **material_params):
    response = gsm_.get_response(u_ta, T_t, t_t, **material_params)
    _t_t, _u_tIa, _T_t, _Eps_tIb, _Sig_tIb, _iter_t, _dF_dEps_t, lam_t = response
    _u_atI, _Eps_btI, _Sig_btI, _dF_dEps_btI = [np.moveaxis(v_, -1, 0) for v_ in (_u_tIa, _Eps_tIb, _Sig_tIb, _dF_dEps_t)]
    _sig_atI = gsm_.get_sig(_u_atI, _T_t, _Eps_btI, _Sig_btI, **material_params )
    return _t_t, _u_atI, _sig_atI, _T_t, _Eps_btI, _Sig_btI, _dF_dEps_btI 

gsm_F.vp_on = True
gsm_F.update_at_k = False
gsm_G.vp_on = True
gsm_G.update_at_k = False

In [ ]:
# params
n_t = 151
n_I = 1
u_T_max = 0.0035
t_t = np.linspace(0, 1, n_t)
u_ta_F = (u_T_max * t_t).reshape(-1, 1)
T_t = 20 + t_t * 0
_t_t_F, _u_atI_F, _sig_atI_F, _T_t_F, _Eps_btI_F, _Sig_btI_F, _dF_dEps_btI_F = gsm_run(gsm_F, u_ta_F, T_t, 0.1*t_t, **material_params)
_max_sig = np.max(_sig_atI_F)
_max_sig
_arg_t_F = _t_t_F[np.argmax(_sig_atI_F)]
_t_F_scale = _arg_t_F * _t_t_F[-1]
_t_F_scale

In [ ]:
# params
t_t = np.linspace(0, 1, n_t)
u_ta_G = (_max_sig * t_t).reshape(-1, 1)
T_t = 20 + t_t * 0
_t_t_G, _u_atI_G, _sig_atI_G, _T_t_G, _Eps_btI_G, _Sig_btI_G, _dF_dEps_btI_G = gsm_run(gsm_G, u_ta_G, T_t, 0.1*t_t, **material_params)
_u_p_atI, _z_atI, _alpha_atI, _omega_atI = gsm_G.Eps_as_blocks(_Eps_btI_F)
_, _Z_atI, _X_atI, _Y_atI = gsm_G.Eps_as_blocks(_Sig_btI_F)


In [ ]:
from scipy.integrate import cumtrapz
fig, ((ax, ax_T, ax_Diss), (ax_omega, ax_3, ax_4)) = plt.subplots(2,3, figsize=(12,6), tight_layout=True)
ax.plot(_u_atI_F[0, :, 0], _sig_atI_F[0, :, 0], label='Helmholtz');
ax.plot(_sig_atI_G[0, :, 0], _u_atI_G[0, :, 0], label='Gibbs');
# ax_T.plot(_t_t_F, _T_t_F);
# ax_T.plot(_t_t_G * _t_F_scale, _T_t_G);
ax.legend()
ax.set_title(r'stress-strain')
ax.set_ylabel(r'$\varsigma$')
ax.set_xlabel(r'$\varepsilon$')

ax_T.plot(_u_atI_F[0, :, 0], _T_t_F, label='Helmholtz');
ax_T.plot(_sig_atI_G[0, :, 0], _T_t_G, label='Gibbs');
ax_T.legend()
ax_T.set_title(r'temperature')
ax_T.set_ylabel(r'$\vartheta$')
ax_T.set_xlabel(r'$\varepsilon$')

Diss_btI_F = cumtrapz(_dF_dEps_btI_F, _Eps_btI_F, initial=0, axis=1)
ax_Diss.plot(_t_t_F, np.sum(Diss_btI_F[...,0], axis=0), alpha=1, label='F')
Diss_btI_G = cumtrapz(_dF_dEps_btI_G, _Eps_btI_G, initial=0, axis=1)
ax_Diss.plot(_t_t_G * _t_F_scale, np.sum(Diss_btI_G[...,0], axis=0), alpha=1, label='G')

ax_omega.plot(_u_atI_F[0, :, 0], _omega_atI[0, :, 0])
ax_omega.set_xlabel(r'$\varepsilon$/-')
ax_omega.set_ylabel(r'$\omega$/-')


## Fatigue loading 

In [ ]:

def generate_cyclic_load(max_s, min_s, freq, total_cycles, points_per_cycle):
    # Calculate the time for one cycle
    total_time = total_cycles / freq

    # Calculate the mean value and amplitude
    mean_value = (max_s + min_s) / 2
    amplitude = (max_s - min_s) / 2

    # Calculate the initial loading slope
    slope = 2 * np.pi * freq * amplitude
    
    # Time arrays for linear increase and sinusoidal part
    initial_duration = mean_value / slope
    initial_points = int(initial_duration * freq * points_per_cycle)
    total_points = int(total_time * freq * points_per_cycle)
    
    # Generate the initial linear increase
    initial_t = np.linspace(0, initial_duration, initial_points, endpoint=False)
    initial_loading = slope * initial_t

    # Generate the sinusoidal loading
    sinusoidal_t = np.linspace(0, total_time, total_points, endpoint=False)
    sinusoidal_loading = mean_value + amplitude * np.sin(2 * np.pi * freq * sinusoidal_t)

    # Combine the initial linear increase with the sinusoidal loading
    t_full = np.concatenate((initial_t, sinusoidal_t + initial_duration))
    s_full = np.concatenate((initial_loading, sinusoidal_loading))
    
    return t_full, s_full

In [ ]:
def arg_max_min(data):
    # Find local maxima and minima
    maxima_indexes = (np.diff(np.sign(np.diff(data))) < 0).nonzero()[0] + 1
    minima_indexes = (np.diff(np.sign(np.diff(data))) > 0).nonzero()[0] + 1
    return maxima_indexes, minima_indexes

In [ ]:
s_1 = sp.Symbol('s_1')
t_1 = sp.Symbol('t_1')
t = sp.Symbol('t')
fn_s_t = sp.Piecewise((t * s_1/t_1, t < t_1),(s_1, True))
get_step_loading = sp.lambdify((t, s_1, t_1), fn_s_t)

In [ ]:
fig, (ax, ax_N) = plt.subplots(2,1, figsize=(8,6))
#t_t, s_t = generate_cyclic_load(max_s=0.66, min_s=0.1, freq=5, total_cycles=10, points_per_cycle=20)
t_t, s_t = generate_cyclic_load(1, 0.1, 0.01, 10, 30)
ax.plot(t_t, s_t, '-o')
arg_max, arg_min = arg_max_min(s_t)
ax.plot(t_t[arg_max], s_t[arg_max], 'o', color='red')
ax.plot(t_t[arg_min], s_t[arg_min], 'o', color='orange')
ax_N.plot(s_t[arg_max], 'o-')
ax_N.plot(s_t[arg_min], 'o-')

u_t_fat = get_step_loading(t_t, s_1=1, t_1=50)
ax.plot(t_t, u_t_fat)


## Step loading

In [ ]:
s_1 = sp.Symbol('s_1')
t_1 = sp.Symbol('t_1')
t = sp.Symbol('t')
fn_s_t = sp.Piecewise((t * s_1/t_1, t < t_1),(s_1, True))
get_step_loading = sp.lambdify((t, s_1, t_1), fn_s_t)

In [ ]:
def sig_p_T_0_lambdified(gsm_):
    sig_p_T_solved_ = sp.solve((gsm_.f_), p1d.sig_p)
    return sp.lambdify((gsm_.T_var, 
                                gsm_.Eps.as_explicit(), 
                                gsm_.Sig.as_explicit()) + gsm_.m_params + ('**kw',), 
                            sig_p_T_solved_, cse=True)
get_sig_p_T_0 = sig_p_T_0_lambdified(gsm_G)

In [ ]:
S_max_levels = np.array([1, 0.95, 0.85, 0.75, 0.65])
#S_max_levels = np.array([1, 0.75])
responses = {}
N_S_min = {}
eta_t_fat = {}

N_S_max = {1: 1}
Diss_Sbt = {1: Diss_btI_F[:,:,0]}
_delta_T = 20
for S_max in S_max_levels[1:]:

    print('S_max', S_max)
    # params
    t_t, s_t = generate_cyclic_load(S_max, 0.1, 5, 1000, 66)
    #t_t, s_t = generate_cyclic_load(1, 0, 0.01, 10, 30)
    u_ta_fat = (_max_sig * s_t).reshape(-1, 1)
    T_t = 20 + t_t * 0

    # u_ta_fat = get_step_loading(t_t, s_1=1, t_1=50).reshape(-1, 1) * _max_sig * S_max

    # T_t = 20 + s_t * _delta_T

    response = gsm_run(gsm_G, u_ta_fat, T_t, t_t, **material_params)

    _t_t_fat, _u_atI_fat, _sig_atI_fat, _T_t_fat, _Eps_btI_fat, _Sig_btI_fat, _dF_dEps_btI_fat = response
    _sig_atI_top, _sig_atI_bot = get_sig_p_T_0(_T_t_fat, _Eps_btI_fat, _Sig_btI_fat, **material_params )
    _u_p_atI, _z_atI, _alpha_atI, _omega_atI = gsm_G.Eps_as_blocks(_Eps_btI_fat)
    _, _Z_atI, _X_atI, _Y_atI = gsm_G.Eps_as_blocks(_Sig_btI_fat)

    _sig_t_fat = _sig_atI_fat[0,:,0]
    arg_max_u, arg_min_u = arg_max_min(_sig_t_fat)
    _N_S_max, _N_S_min = len(arg_max_u), len(arg_min_u)
    N_S_max[S_max] = _N_S_max
    N_S_min[S_max] = _N_S_min

    _eta_t_fat = _t_t_fat / _t_t_fat[-1]
    eta_t_fat[S_max] = _eta_t_fat
    responses[S_max] = response

    Diss_btI_fat = cumtrapz(_dF_dEps_btI_fat, _Eps_btI_fat, initial=0, axis=1)
    Diss_Sbt[S_max] = Diss_btI_fat[:, :, 0]

_N_S_max = np.array([N_S_max[key] for key in S_max_levels])
_Diss_plastic_St = {S_max : np.sum(Diss_Sbt[S_max][:-1, :], axis=0) for S_max in S_max_levels}
_Diss_damage_St = {S_max : Diss_Sbt[S_max][-1, :] for S_max in S_max_levels}

_Diss_plastic_S = np.array([_Diss_plastic_St[S_max][-1] for S_max in S_max_levels])
_Diss_damage_S = np.array([_Diss_damage_St[S_max][-1] for S_max in S_max_levels])

In [ ]:
from bmcs_utils.api import mpl_align_yaxis_to_zero

colors = ['black', 'red', 'darkslategrey', 'maroon', 'darkblue', 'magenta']

# plt.rcParams.update({
#     "font.family": "serif",
#     "font.serif": ["Computer Modern Roman"]
# })

def get_fig( include_last_col=True ):
    if include_last_col:
        fig, ((ax_sig_u, ax_el, ax_T), (ax_omega, ax_u, ax_SN)) = plt.subplots(2,3, figsize=(12,7), tight_layout=True)
    else:
        fig, ((ax_sig_u, ax_el), (ax_omega, ax_u)) = plt.subplots(2,2, figsize=(12,7), tight_layout=True)

    fig.canvas.header_visible=False

    ax_sig_u.set_title('stress-strain')
    ax_sig_u.plot(_u_atI_F[0,:,0], _sig_atI_F[0,:,0], color='black', ls='dashed')
    ax_sig_u.plot(_sig_atI_G[0,:,0], _u_atI_G[0, :,0], color='black')

    ax_omega.set_title('damage')
    ax_el.set_title('elastic domain')
    ax_u.set_title('fatigue strain')

    ax_sig_u.set_xlabel(r'$\varepsilon$/-')
    ax_sig_u.set_ylabel(r'$\sigma$/MPa')
    ax_omega.set_xlabel(r'$\eta$/-')
    ax_omega.set_ylabel(r'$\omega$/-')
    ax_el.set_xlabel(r'$\eta$/-')
    ax_el.set_ylabel(r'$\sigma$/MPa')
    ax_u.set_xlabel(r'$\eta$/-')
    ax_u.set_ylabel(r'$\varepsilon$/-')

    if include_last_col:
        ax_T.set_title('temperature')
        ax_SN.set_title('S-N & dissipation')

        ax_T.set_xlabel(r'$\eta$/-')
        ax_T.set_ylabel(r'$\vartheta$/$^{\circ}$C')

        ax_SN.set_xlabel(r'log$N$/-')
        ax_SN.set_ylabel(r'$S_\mathrm{max}$/-')

        return fig, ((ax_sig_u, ax_el, ax_T), (ax_u, ax_omega, ax_SN)) 

    else:
        return fig, ((ax_sig_u, ax_el), (ax_u, ax_omega)) 

def plot_response(S_max_levels, responses, N_S_max, S_max_select=None, include_last_col=True):
    if S_max_select is None:
        S_max_select = np.arange(len(S_max_levels))
    fig_ax = get_fig(include_last_col)

    if include_last_col:
        fig, ((ax_sig_u, ax_el, ax_T), (ax_u, ax_omega, ax_SN)) = fig_ax
        ax_SN_twin = ax_SN.twinx()
        ax_SN_twin.set_ylabel(r'$\mathcal{D}/$J/mm$^{{2}}$')
    else:
        fig, ((ax_sig_u, ax_el), (ax_u, ax_omega)) = fig_ax

    alpha_line = 0.5

#    for i, (S_max, response) in enumerate(responses.items()):
    for i in S_max_select:
        S_max = S_max_levels[i+1]
        response = responses[S_max]
        c = colors[i+1]
        # params
        _t_t_fat, _u_atI_fat, _sig_atI_fat, _T_t_fat, _Eps_btI_fat, _Sig_btI_fat, _dF_dEps_btI_fat = response
        _sig_atI_top, _sig_atI_bot = get_sig_p_T_0(_T_t_fat, _Eps_btI_fat, _Sig_btI_fat, **material_params )
        _u_p_atI, _z_atI, _alpha_atI, _omega_atI = gsm_G.Eps_as_blocks(_Eps_btI_fat)
        _, _Z_atI, _X_atI, _Y_atI = gsm_G.Eps_as_blocks(_Sig_btI_fat)

        _sig_t_fat = _sig_atI_fat[0,:,0]
        arg_max_u, arg_min_u = arg_max_min(_sig_t_fat)
        _eta_t_fat = eta_t_fat[S_max]

        _sig_t_fat = _sig_atI_fat[0,:,0]
        _u_t_fat = _u_atI_fat[0,:,0]
        ax_sig_u.plot(_sig_t_fat, _u_t_fat, color=c, alpha=alpha_line, label=f'$S_{{\max}} = {S_max}$' )

        ax_omega.plot(_eta_t_fat, _omega_atI[0, :, 0], color=c, lw=2, label=f'$N = {N_S_max[S_max]}$' )

        ax_el.plot(_eta_t_fat, _sig_atI_top[:, 0], color=c, alpha=alpha_line)
        ax_el.plot(_eta_t_fat, _sig_atI_bot[:, 0], color=c, alpha=alpha_line)
#        ax_el.plot(_eta_t_fat, _u_atI_fat[0, :, 0], color=c, lw=0.5)
        ax_el.fill_between(_eta_t_fat, _sig_atI_bot[:, 0], _sig_atI_top[:, 0], color=c, alpha=0.1)

        _eta_max_n = np.linspace(0, 1, N_S_max[S_max])
        _eta_min_n = np.linspace(0, 1, N_S_min[S_max])

        ax_u.plot(_eta_max_n, _sig_t_fat[arg_max_u], '-', lw=2, color=c)
        ax_u.plot(_eta_min_n, _sig_t_fat[arg_min_u], '--', lw=2, color=c)

        if include_last_col:
            ax_T.plot(_eta_t_fat, _T_t_fat, lw=2, color=c)

        arg_dissip = np.where( _dF_dEps_btI_fat[-1,:,0] > 1e-4 )
        # ax_sig_u.plot(_sig_t_fat[arg_dissip], _u_t_fat[arg_dissip], 'o', color='yellow', markersize=2)
        # ax_el.plot(_eta_t_fat[arg_dissip], _u_t_fat[arg_dissip], 'o', color='orange', markersize=3)
        # ax_el.plot(_eta_t_fat, _u_t_fat, 'o', color='yellow', markersize=2)

    if include_last_col:
        _extracted_from_plot_response_49(ax_SN, S_max_levels, ax_SN_twin)
    ax_u.set_ylim(ymin=0)

    ax_sig_u.legend()
    ax_omega.legend()
    #ax_el.set_ylim(ymax=_max_sig)
    return fig


# TODO Rename this here and in `plot_response`
def _extracted_from_plot_response_49(ax_SN, S_max_levels, ax_SN_twin):
    ax_SN.semilogx(_N_S_max, S_max_levels, 'o-', lw=2, label='S-N')
    ax_SN.legend()

    ax_SN_twin.semilogx(_N_S_max, _Diss_plastic_S, 'o-', alpha=0.4, color='red', label='plastic')
    ax_SN_twin.semilogx(_N_S_max, _Diss_damage_S, 'o-', alpha=0.4, color='gray', label='damage')
    ax_SN_twin.legend()
    ax_SN_twin.set_xscale('log')


from pathlib import Path
path = Path().home() / 'simdb' / 'data'
plot_config = {
#     'one': ([0], True),
    # 'two': ([1,3], False),
    # 'endurance' : ([1,4], False), 
    # 'three': ([1,2, 3], False),
    # 'four': ([0, 1,2, 3], False),
     'all': ([0, 1,2, 3], True),
}

for name, config in plot_config.items():
    S_max_select, include_last_col = config
    fig = plot_response(S_max_levels, responses, N_S_max, S_max_select=S_max_select, include_last_col=include_last_col);
    fname = f'GSM_demo_fatigue_uniaxial_stress_{name}.png'
    fig.savefig(path / fname)



## Lessons

- What is the formula for an endurance limit of S_max?

In [ ]:
sp.simplify(gsm_G.Phi)

## Animation

In [ ]:
N_S_max = [1]
Diss_Sb = [Diss_btI_F[:,-1,0]]

alpha_line = 0.7

S_index = 3
_S_max = S_max_levels[S_index] 
response = responses[_S_max]

_t_t_fat, _u_atI_fat, _sig_atI_fat, _T_t_fat, _Eps_btI_fat, _Sig_btI_fat, _dF_dEps_btI_fat = response
_sig_atI_top, _sig_atI_bot = get_sig_p_T_0(_T_t_fat, _Eps_btI_fat, _Sig_btI_fat, **material_params )
_u_p_atI, _z_atI, _alpha_atI, _omega_atI = gsm_G.Eps_as_blocks(_Eps_btI_fat)
_, _Z_atI, _X_atI, _Y_atI = gsm_G.Eps_as_blocks(_Sig_btI_fat)

_eta_t_fat = eta_t_fat[_S_max]
_c = colors[S_index]

# Initialize the data arrays
x_sig_u_95 = _sig_atI_fat[0, :, 0]
y_sig_u_95 = _u_atI_fat[0, :, 0]

_Diss_plastic_t = _Diss_plastic_St[_S_max]
_Diss_damage_t = _Diss_damage_St[_S_max]
_Diss_t = _Diss_damage_t + _Diss_plastic_t
_dDiss_dt = np.diff(_Diss_t)



In [ ]:
import numpy as np
from matplotlib.collections import LineCollection
import matplotlib.pyplot as plt
y = np.copy(y_sig_u_95)
x = np.copy(x_sig_u_95)
#lwidths=0+x[:-1]
lwidths = np.array( _dDiss_dt > 0, dtype=np.int_ ) * 3
points = np.array([x, y]).T.reshape(-1, 1, 2)
segments = np.concatenate([points[:-1], points[1:]], axis=1)
fig,a = plt.subplots()
line, = a.plot(x, y, '-', color='darkblue', alpha=0.3)
lc = LineCollection(segments, linewidths=lwidths,color='red')
a.add_collection(lc)
n_s = -1
line.set_data(x[:n_s], y[:n_s])
lc.set_segments(segments[:n_s])

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation

fig, ((ax_sig_u, ax_el), (ax_u, ax_omega)) = get_fig(include_last_col=False)

ax_diss = ax_u.twinx()
ax_diss.set_ylabel(r'$\mathcal{D} / \mathrm{J}\cdot \mathrm{mm}^{-2}$')
#colors = ['black', 'blue', 'chocolate', 'gray', 'brown']

_c = colors[S_index]
_cursor_color = 'crimson'

ax_sig_u.plot(_u_atI_F[0,:,0], _sig_atI_F[0,:,0], color='black', ls='dashed')
ax_sig_u.plot(_sig_atI_G[0,:,0], _u_atI_G[0, :,0], color='black')
ax_omega.plot(_eta_t_fat, _omega_atI[0, :, 0], color='white')
ax_el.fill_between(_eta_t_fat, _sig_atI_bot[:, 0], _sig_atI_top[:, 0], color='white', alpha=0.1)
ax_u.plot(_eta_t_fat, _sig_atI_fat[0,:,0], '-', color='white', alpha=alpha_line)

ax_diss.plot(_eta_t_fat, _Diss_t, color='white')

# Set up empty plot elements that we will update
sig_u_line, = ax_sig_u.plot([], [], color=_c, lw=2, alpha=0.4)
sig_u_line_cycle, = ax_sig_u.plot([], [], color=_c, lw=2)
t_sig_line, = ax_el.plot([], [], color=_c, lw=2, ls='solid')
t_sig_top, = ax_el.plot([], [], color=_c, lw=1, alpha=alpha_line)
t_sig_bot, = ax_el.plot([], [], color=_c, lw=1, alpha=alpha_line)
t_sig_bar, = ax_el.plot([], [], color=_cursor_color, lw=3)
t_omega_line, = ax_omega.plot([], [], color=_c)
t_u_line, = ax_u.plot([], [], color=_c)
t_diss_plastic_line, = ax_diss.plot([], [], color='firebrick', alpha=0.7)
t_diss_damage_line, = ax_diss.plot([], [], color='gray', alpha=0.7)

_markersize = 10
sig_u_point, = ax_sig_u.plot([], [], 'o', color=_cursor_color, markersize=_markersize)
t_sig_point, = ax_el.plot([], [], 'o', color=_cursor_color, markersize=_markersize)
t_omega_point, = ax_omega.plot([], [], 'o', color=_cursor_color, markersize=_markersize)
t_u_point, = ax_u.plot([], [], 'o', color=_cursor_color, markersize=_markersize)

t_sig_fill = ax_el.fill_between([], [], color=_c, alpha=0.1)
t_diss_damage_fill = ax_diss.fill_between([], [], color='gray', alpha=0.2)
t_diss_plastic_fill = ax_diss.fill_between([], [], color='firebrick', alpha=0.2)

# Initialization function to set up the background of the plot
def init():
    _init_lines(sig_u_line, sig_u_line_cycle, sig_u_point, t_sig_line, 
                t_sig_point, t_sig_bot, t_sig_top, t_sig_bar,
                t_omega_line, t_u_line, t_u_point, t_diss_damage_line, t_diss_plastic_line)
    
    global t_sig_fill, t_diss_damage_fill, t_diss_plastic_fill
    t_sig_fill.remove()  # Remove the initial empty fill
    t_diss_damage_fill.remove()
    t_diss_plastic_fill.remove()
    t_sig_fill = ax_el.fill_between([], [], [], color=_c, alpha=0.1)
    t_diss_damage_fill = ax_diss.fill_between([], [], [], color='gray', alpha=0.2)
    t_diss_plastic_fill = ax_diss.fill_between([], [], [], color='firebrick', alpha=0.2)

    return (sig_u_line, sig_u_point, t_sig_line, t_sig_point, 
            t_sig_fill, t_omega_line, t_u_line, t_u_point,
            t_diss_damage_line, t_diss_plastic_line)

def _init_lines(*args):
    for arg in args:
        arg.set_data([], [])

# Animation function which updates the plot for each frame
def replot(i):
    # Update the line up to the ith point
    # print('frame', i, x_sig_u_95[:i].shape, y_sig_u_95[:i].shape)
    print(f'{i}', end='\r')
    sig_u_line.set_data(x_sig_u_95[:i], y_sig_u_95[:i])
    min_i_cycle = i - 30
    min_i_cycle = max(min_i_cycle, 0)
    sig_u_line_cycle.set_data(x_sig_u_95[min_i_cycle:i], y_sig_u_95[min_i_cycle:i])
    t_sig_line.set_data(_eta_t_fat[:i], y_sig_u_95[:i])

    t_sig_top.set_data(_eta_t_fat[:i], _sig_atI_top[:i, 0])
    t_sig_bot.set_data(_eta_t_fat[:i], _sig_atI_bot[:i, 0])

    x_data = _eta_t_fat[:i]
    top_data = _sig_atI_bot[:i, 0]
    bot_data = _sig_atI_top[:i, 0]

    global t_sig_fill, t_diss_damage_fill, t_diss_plastic_fill
    top_data = _sig_atI_bot[:i, 0]
    bot_data = _sig_atI_top[:i, 0]
    t_sig_fill.remove()  # Remove the initial empty fill
    t_sig_fill = ax_el.fill_between(x_data, top_data, bot_data, color=_c, alpha=0.1)
    top_data = _Diss_damage_t[:i]
    t_diss_damage_fill.remove()
    t_diss_damage_fill = ax_diss.fill_between(x_data, top_data, np.zeros_like(top_data), color='gray', alpha=0.2)
    top_data = _Diss_t[:i]
    bot_data = _Diss_damage_t[:i]
    t_diss_plastic_fill.remove()
    t_diss_plastic_fill = ax_diss.fill_between(x_data, top_data, bot_data, color='firebrick', alpha=0.2)

    # global t_sig_fill
    # t_sig_fill.remove()  # Remove the old fill
    # t_sig_fill = ax_el.fill_between(x_data, top_data, bot_data, color=_c, alpha=0.1)

    t_omega_line.set_data(_eta_t_fat[:i], _omega_atI[0, :i, 0])
    t_u_line.set_data(_eta_t_fat[:i], _sig_atI_fat[0,:i,0])

    t_diss_damage_line.set_data(_eta_t_fat[:i], _Diss_damage_t[:i])
    t_diss_plastic_line.set_data(_eta_t_fat[:i], _Diss_t[:i])

    # Update the point to the ith point
    if i > 0:
        _plot_current_point(i)
    return (sig_u_line, sig_u_point, t_sig_line, t_sig_point, 
            t_sig_fill, t_omega_line, t_u_line, t_u_point, t_diss_damage_line, t_diss_plastic_line)


def _plot_current_point(i):
    # TODO Rename this here and in `replot`
    i1 = i-1
    sig_u_point.set_data(x_sig_u_95[i1], y_sig_u_95[i1])
    t_sig_point.set_data(_eta_t_fat[i1], y_sig_u_95[i1])
    t_sig_bar.set_data([_eta_t_fat[i1],_eta_t_fat[i1]], [_sig_atI_bot[i1, 0], _sig_atI_top[i1, 0]])
    t_omega_point.set_data(_eta_t_fat[i1], _omega_atI[0, i1, 0])
    t_u_point.set_data(_eta_t_fat[i1], _sig_atI_fat[0,i1,0])


In [ ]:
if False:
    #print(f'{i}', end='\n')

    # Number of frames for the animation
    num_frames = len(x_sig_u_95)
    frame_interval = 500

    # # Create the animation object
    ani = animation.FuncAnimation(fig, replot, init_func=init,
                                frames=num_frames, interval=300, blit=True, 
                                repeat=False)

    output_file = Path().home() / 'simdb' / 'data' / 'GSM_demo_anim.mp4'
    ani.save(output_file);

    # output_file = Path().home() / 'simdb' / 'data' / 'GSM_demo_anim.gif'
    # ani.save(output_file);

    #from PIL import Image

**Conclusion:**
To link the structural behavior with changes of the microstructure it is necessary to capture irreversibility of the structural response using thermodynamics. Frequently used ad hoc additions to damage models are insufficient. A necessary condition for a cycle-by-cycle simulation of fatigue is for a model is its ability to capture irreversible changes of internal state both at the upper and lower load limit.


In [ ]:

normed_dDiss_dt = _dDiss_dt / np.max(_dDiss_dt) 
# lwidths = np.array( _dDiss_dt > 0, dtype=np.int_ ) * 3
lwidths = np.copy( _dDiss_dt)
lwidths[lwidths > 0] = 5

In [ ]:
np.max(lwidths)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from matplotlib.collections import LineCollection

fig, ax_sig_u = plt.subplots()
fig.canvas.header_visible=False

ax_sig_u.set_title('stress-strain')
ax_sig_u.plot(_u_atI_F[0,:,0], _sig_atI_F[0,:,0], lw=1, color='black', ls='dashed')
ax_sig_u.plot(_sig_atI_G[0,:,0], _u_atI_G[0, :,0], lw=1, color='black')

_c = colors[S_index]
_c = 'darkblue'
_cursor_color = 'crimson'
_cursor_color = 'darkblue'

ax_sig_u.plot(_u_atI_F[0,:,0], _sig_atI_F[0,:,0], color='black', ls='dashed')
ax_sig_u.plot(_sig_atI_G[0,:,0], _u_atI_G[0, :,0], color='black')

# Set up empty plot elements that we will update
sig_u_line, = ax_sig_u.plot([], [], color=_c, lw=1, alpha=0.3)
sig_u_line_cycle, = ax_sig_u.plot([], [], color=_c, lw=2)

_markersize = 10
sig_u_point, = ax_sig_u.plot([], [], 'o', color=_cursor_color, markersize=_markersize)

# Create initial data
x = x_sig_u_95
y = y_sig_u_95

# Create segments for LineCollection
points = np.array([x, y]).T.reshape(-1, 1, 2)
segments = np.concatenate([points[:-1], points[1:]], axis=1)

# Create a figure and axis
line_collection = LineCollection(segments, linewidths=1, color='red')
ax_sig_u.add_collection(line_collection)

def init():
    _init_lines(sig_u_line, sig_u_line_cycle, sig_u_point)
    line_collection.set_segments([])  # Initialize with empty segments
    return line_collection, sig_u_line, sig_u_line_cycle, sig_u_point

def _init_lines(*args):
    for arg in args:
        arg.set_data([], [])

def animate(i):
    print(f'{i}', end='\r')

    sig_u_line.set_data(x[:i], y[:i])
    n_cycle = 66
    min_i_cycle = i - n_cycle
    min_i_cycle = max(min_i_cycle, 0)
    sig_u_line_cycle.set_data(x[min_i_cycle:i], y[min_i_cycle:i])
    # Update the segments to show only up to the current index 'i'
    current_segments = segments[:i]
    current_widths = np.copy(lwidths[:i])
    current_widths[:(-min_i_cycle-1)] *= 0.5
    line_collection.set_segments(current_segments)    
    line_collection.set_linewidths(current_widths)

    if i > 0:
        _plot_current_point(i)

    return line_collection, sig_u_line, sig_u_line_cycle, sig_u_point



def _plot_current_point(i):
    # TODO Rename this here and in `replot`
    #i1 = i-1
    sig_u_point.set_data(x[i], y[i])

# Set the number of frames and interval between frames
num_frames = len(x)
frame_interval = 13  # in milliseconds

ani = FuncAnimation(fig, animate, frames=num_frames, interval=frame_interval, blit=True, init_func=init)
output_file = Path().home() / 'simdb' / 'data' / 'GSM_demo_anim.mp4'
ani.save(output_file);